<a href="https://colab.research.google.com/github/karthikeyen-creations/GENAI/blob/main/Assignment2/classification_tasks_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai==1.23.2 datasets==2.19.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
import json
import numpy as np

from openai import AzureOpenAI
from datasets import load_dataset
from sklearn.metrics import classification_report
from google.colab import userdata
from tqdm import tqdm

In [ ]:
azure_api_key = userdata.get('azure_api_key')
azure_endpoint = userdata.get('azure_endpoint')

In [ ]:
client = AzureOpenAI(
  azure_endpoint = azure_api_endpoint,
  api_key=azure_api_key,
  api_version="2024-02-01"
)

In [ ]:
model_name = 'gpt-35-turbo' # deployment name

**Examples and Gold Examples**

A set of examples and gold examples for sentiment classification of Amazon product reviews is hosted in a HuggingFace dataset. Let us load this data and take a look at the samples in this data.

In [ ]:
amazon_reviews = load_dataset("pgurazada1/amazon_polarity")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating examples split:   0%|          | 0/32 [00:00<?, ? examples/s]

Generating gold_examples split:   0%|          | 0/32 [00:00<?, ? examples/s]

As is evident from the above output, the data set has 32 samples as examples and 32 samples as gold examples.

In [ ]:
amazon_reviews_examples_df = amazon_reviews['examples'].to_pandas()
amazon_reviews_gold_examples_df = amazon_reviews['gold_examples'].to_pandas()

In [ ]:
amazon_reviews_examples_df.shape, amazon_reviews_gold_examples_df.shape

((32, 2), (32, 2))

As the above outputs indicate, there are 32 examples and 32 gold examples. We will sample from the examples to create the few shot prompt and evaluate the prompt on all 32 gold examples.

In [ ]:
amazon_reviews_examples_df.sample(4)

,content,label
24,"Of Course Of course this album was hella ill, ...",1
26,We moved to Budapest and will travel to other ...,0
7,The COPS(Central Command Police Specialists) a...,0
30,Avoid this line of Sony portable CD players - ...,0


**Assembling the prompt**

In [ ]:
system_message = """
Classify product reviews in the input as positive or negative in sentiment.
Do not explain your answer. Your answer should only contain the label: 1 (positive) or 0 (negative).
"""

In [ ]:
few_shot_prompt = [{'role':'system', 'content': system_message}]

We need to iterate over the rows of the examples DataFrame to append these examples as `user` and `assistant` messages to the few-shot prompt. We achieve this using the `iterrows` method.

In [ ]:
for index, row in amazon_reviews_examples_df.iterrows():
    print('Example Review:')
    print(row[0])
    print('Example Label:')
    print(row[1])
    break

Example Review:
i bought three of these and had problems with all three! as another reviewer noted, any time you touch the wire, this thing goes off and i have to manually reset which causes an odd buzzing that i don't like the sound of. i have this under my computer desk and occasionally hit the wire with my feet. a surge protector should be more robust than this and since i have had the same problem with three units ( i bought them all at once or would never repurchase)i don't believe it is a rare defect but built into the design.
Example Label:
0


Notice that the label is an integer. However, LLMs accept only strings. So we need to convert the integer label to a string label as we assemble the few-shot prompt. Let us assemble a few-shot prompt with 4 examples.

In [ ]:
for index, row in amazon_reviews_examples_df.sample(4).iterrows():
    example_review = row[0]
    example_label = row[1]

    few_shot_prompt.append(
        {
            'role': 'user',
            'content': example_review
        }
    )

    few_shot_prompt.append(
        {
            'role': 'assistant',
            'content': str(example_label) # LLMs accept only string inputs
        }
    )

In [ ]:
few_shot_prompt

[{'role': 'system',
  'content': '\nClassify product reviews in the input as positive or negative in sentiment.\nDo not explain your answer. Your answer should only contain the label: 1 (positive) or 0 (negative).\n'},
 {'role': 'user',
  'content': 'The COPS(Central Command Police Specialists) are a group of the best crime fighters in the U.S. led by fed agent V.P. Vess codename Bulletproof which are formed to fight against The Big Boss and his henchmen in Empire City the nation most corrupted and criminilized city.These 3 episodes contained in this disc are the 3 first produced by DIC animation in the 1988 season.The opening credits are omited and only the closing credits are included after the last ep.No special feats are included nor captions or subtitles following the standard in these low budget transfers.The audio and picture quality is good and that is the only good thing in this disc.We deserve more eps and these released in the way of a complete season like G.I. Joe and other

We now have 4 examples in the few shot prompt that is ready for use. Before we deploy this prompt, we need to get an estimate of the performance of this prompt. Here is where we use gold examples to estimate the accuracy.

## Evaluation

In [ ]:
predictions, ground_truths = [], []

In [ ]:
for index, row in tqdm(amazon_reviews_gold_examples_df.iterrows()):
    gold_review = row[0]
    gold_label = row[1]

    user_input = [{'role':'user', 'content': gold_review}]

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=few_shot_prompt + user_input,
            temperature=0
        )

        predictions.append(int(response.choices[0].message.content)) # convert the string label back to int
        ground_truths.append(gold_label)
    except Exception as e:
        print(e) # Log error and continue
        continue

15it [00:04,  3.62it/s]

Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': True, 'severity': 'high'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}


32it [00:08,  3.56it/s]


In [ ]:
predictions = np.array(predictions)
ground_truths = np.array(ground_truths)
(predictions == ground_truths).mean()

0.9354838709677419

The output above indicates that the accuracy of the few-shot prompt on gold examples. More fine-grained evaluation (e.g., F1 score) could also be used to establish the estimated accuracy of the prompt.

In [ ]:
print(classification_report(ground_truths, predictions))

              precision    recall  f1-score   support

           0       1.00      0.89      0.94        18
           1       0.87      1.00      0.93        13

    accuracy                           0.94        31
   macro avg       0.93      0.94      0.93        31
weighted avg       0.94      0.94      0.94        31



>More examples does not imply better accuracy. Increasing the number of examples in the few-shot prompt beyond 16 is not known to yield better performance.